# Protein Tinder
## Find the Protein of your dreams

## Documentation
Edit parameters in dependencies:
1. directory - Directory where input pdbs are placed. Input can contain subdirectories.
2. switchIdent - Pattern matching string to identify switched state pdb.
3. outDir - intermediate dictionary, where centered and aligned pdbs are stored.
4. finalDir - final directory where selected pdbs are stored.  

If the intermediate (centered and rotated) pdbs exist, do not run the "Align switches" cells, they will be overwritten.


## Utility functions

In [ ]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 50
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))
    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

## Import dependencies

In [ ]:
import py3Dmol as p3d
import shutil
import time
import numpy as np
import os
from itertools import chain
from glob import glob
import Bio.PDB as bpdb

directory = "2023-05-01_Switch"
switchIdent = "_SWITCH"
outdir = "CenterAligned"
finaldir = "out"

## Load pdbs

In [ ]:
def enumDir(directory):
    files = list((chain.from_iterable(glob(os.path.join(x[0], '*.pdb')) for x in os.walk(directory))))
    print(files)

    """
    switched file must be original file name with the addition of switchIdent anywhere in the filename
    """

    switchedStates = list(filter(lambda x: switchIdent in x.split("\\")[-1], files))
    relaxedState = list(filter(lambda x: switchIdent not in x.split("\\")[-1], files))

    proteins = {}

    for rel in relaxedState: 
        namePdb = rel.split("\\")[-1]
        correspondingSwitch = list(filter(lambda x: namePdb in x.replace(switchIdent, ""), switchedStates))
        assert len(correspondingSwitch) == 1
        protein = {"relaxed":{"path":rel}, "switched":{"path":correspondingSwitch[0]}}
        proteins[namePdb.replace(".pdb", "")] = protein
    return proteins

protiens = enumDir(directory=directory)

## Align Switches
Dont run if intermediate pdbs exist. If directory already exists, it will be overwritten!!

In [ ]:
"""Alignement mehtods"""

def getAlignementAtoms(model, alignedModel,start, residueEnd):
    #Alignement based on start:finish residues
    ref_atoms = []
    sample_atoms = []
    atoms_to_be_aligned = range(start, residueEnd)
    
    ref_model = alignedModel
    for ref_chain in [ref_model["A"]]:
        for ref_res in ref_chain:
          if ref_res.get_id()[1] in atoms_to_be_aligned:
            ref_atoms.append(ref_res['CA'])

    sample_model = model
    for sample_chain in [sample_model["A"]]:
      for sample_res in sample_chain:
        if sample_res.get_id()[1] in atoms_to_be_aligned:
          sample_atoms.append(sample_res['CA'])
    assert len(sample_atoms) == len(ref_atoms)
    return sample_atoms, ref_atoms

def alignFirstResidues(pdb, alignedPdb, residueStart,  residueEnd):
    parserAligned = bpdb.PDBParser(QUIET=True)
    parser = bpdb.PDBParser(QUIET=True)
    modelAligned = parserAligned.get_structure("aligned",alignedPdb)[0]
    model = parser.get_structure("toalign", pdb)[0]
    sample_atoms, ref_atoms = getAlignementAtoms(model=model, alignedModel=modelAligned,start=residueStart, residueEnd=residueEnd)
    superImposer = bpdb.Superimposer()
    superImposer.set_atoms(ref_atoms, sample_atoms)
    #print(list(model.get_atoms()))
    superImposer.apply(model.get_atoms())
    
    com = modelAligned["A"].center_of_mass()
    modelAligned.transform(np.eye(3), -com)
    model.transform(np.eye(3), -com)
    return model.get_parent(), modelAligned.get_parent()

In [ ]:
io=bpdb.PDBIO()
for n, (k, v) in enumerate(proteins.items()):
    update_progress(n / len(proteins.keys()))
    switchedPdb = v["switched"]["path"]
    relaxedPdb = v["relaxed"]["path"]
    switchedStructure, relaxedStructure = alignFirstResidues(switchedPdb, relaxedPdb,residueStart=10, residueEnd= 75) #change if necessary
    switchedModel = switchedStructure[0]
    for chain in switchedModel:
        chain.id = "C"
    io.set_structure(switchedStructure)
    io.save(f"{outdir}\\{k.replace('_', '_SWITCH_')}.pdb")
    io.set_structure(relaxedStructure)
    io.save(f"{outdir}\\{k}.pdb")
update_progress(1)

## Display
If prompt wants you to give input, write "y" or "yay" to accept structures. If structure did not load, wait a few seconds, then input "reload" to try again. If anything other is input, the structure is discarded!!

In [ ]:
cAProteins = enumDir(outdir)

valid = {}

for n, (k,v) in enumerate(cAProteins.items()):
    inp = "reload"
    while inp == "reload":
        with open(v["switched"]["path"], "r") as f:
            contentsS = f.read()

        with open(v["relaxed"]["path"], "r") as f:
            contentsR = f.read()

        view = p3d.view(height=400, width=1200, viewergrid=(1,3))
        clear_output(wait = False)
        view.addModel(contentsS,'pdb', viewer=(0,0))
        view.addModel(contentsR,'pdb', viewer=(0,1))
        view.addModel(contentsS,'pdb', viewer=(0,2))
        view.addModel(contentsR,'pdb', viewer=(0,2))
        
        view.setStyle({'cartoon': {'colorscheme':'chainHetatm'}})
        view.setStyle({'chain':"A"},{'cartoon': {'color':"red"}}) # color of switched protein
        view.setStyle({'chain':"B"},{'cartoon': {'color':"yellow"}}) # color of actuator protein
        view.setStyle({'chain':"C"},{'cartoon': {'color':"blue"}}) # color of relaxed protein
        view.zoomTo()
        view.show()
        
        time.sleep(1.5) # Don't edit, it works, don't ask me why
        inp = input("yay(y) or nay(n)?")
        if inp == "y" or inp == "yay":
            valid[k] = v
        time.sleep(.5) # Don't edit this eighter
        del view
        time.sleep(.5) # Same here
    


In [ ]:
print(valid)
for n, (k, v) in enumerate(valid.items()):
    update_progress(n/len(valid.keys()))
    os.makedirs(os.path.dirname(f'{finaldir}\\{v["switched"]["path"]}'), exist_ok=True)
    shutil.copyfile(v["switched"]["path"], f'{finaldir}\\{v["switched"]["path"]}')
    shutil.copyfile(v["relaxed"]["path"], f'{finaldir}\\{v["relaxed"]["path"]}')
update_progress(1)